# Sistem Criptográfico de Chave Púbica de Rabin


### Grupo:
 
 - João Marques, a84684
 - Luis Bigas,   a76964

# Gerar Chave:

In [1]:
while 1:
    p = random_prime(2^128, 2^127, True)
    # p == 3 mod 4 ou p == 5 mod 8
    if (mod(p,4) == 3) or (mod(p,8) == 5):
        break

while 1:
    q = random_prime(2^127, 2^126, True)
    # q == 3 mod 4 ou q == 5 mod 8
    if (mod(q,4) == 3) or (mod(q,8) == 5):
        break

print("p: ", p ,"\n")

print("q: ", q )

p:  207609517445883285549075577486420649279 

q:  34383223814054637056137416917838528671


$n = p \times q$

In [2]:
n = p * q
print("n: ", n , "\n")

n:  7138284504269685812814890223560267091179245078270971466881304280429876978209 



Chave Pública: $PubKey = n$

Chave Privada: $PrivKey = (p,q)$

In [3]:
PubKey = n
PrivKey = (p,q)
print("PubKey:",n, "\nPrivKey:", (p,q))

PubKey: 7138284504269685812814890223560267091179245078270971466881304280429876978209 
PrivKey: (207609517445883285549075577486420649279, 34383223814054637056137416917838528671)


# Cifração: 

In [4]:
msg=1234
print("Mensagem: ",msg)

Mensagem:  1234


In [5]:
msg = int(str(msg) + str(msg))
msg

12341234

In [6]:
msg < PubKey and msg > 0

True

$cifmsg \equiv msg^{2} \mod n$

In [7]:
Zn = IntegerModRing(n)

In [8]:
cifmsg =  Zn(msg^2)
print("Mensagem Cifrada: ", cifmsg)

Mensagem Cifrada:  152306056642756


# Decifração:

$msg^2 \equiv cif \mod n$

$\iff$

$msg \equiv \sqrt{cif} \mod n \vee -msg \equiv \sqrt{cif} \mod n$

Se $p \equiv 3 \mod 4$:

$r = c^{\frac{(p+1)}{4}}\mod p$

Caso $p \equiv 5 \mod 8$:

* Calcular: $d=c^{\frac{p-1}{4}} \mod p$
    * Se $d=1$:
        * $r=c^{\frac{p+3}{8}} \mod p$
    * Se $d=p-1$:
        * $r = 2 c (4 c^{\frac{p-5}{8}}) \mod p$

In [9]:
def sqrt_mod(p,c):
    if(mod(p,4) == 3):
        Zp = IntegerModRing(p)
        rp = c^((p+1)/4)
        r = Zp(rp)
        return r
    if(mod(p,8) == 5):
        Zp = IntegerModRing(p)
        dp = c^((p-1)/4)
        d = Zp(dp)
        if(d == 1):
            rp = c^((p+3)/8)
            r = Zp(rp)
            return r
        if(d == p-1):
            rp = 2*c*((4*c)^((p-5)/8))
            r = Zp(rp)
            return r

In [10]:
r = sqrt_mod(p,cifmsg)
s = sqrt_mod(q,cifmsg)
r,p

(12341234, 207609517445883285549075577486420649279)

  \begin{cases}
    m \equiv r \mod p\\
    m \equiv s \mod q
  \end{cases}

In [11]:
m1 = crt(int(r),int(s),int(p),int(q))
m1

12341234

  \begin{cases}
    m \equiv -r \mod p\\
    m \equiv s \mod q
  \end{cases}

In [12]:
m2 = crt(int(-r),int(s),int(p),int(q))
m2

6125474802395748492703303010173044555221739825019323676217505696736377292704

  \begin{cases}
    m \equiv r \mod p\\
    m \equiv -s \mod q
  \end{cases}

In [13]:
m3 = crt(int(r),int(-s),int(p),int(q))
m3

1012809701873937320111587213387222535957505253251647790663798583693499685505

  \begin{cases}
    m \equiv -r \mod p\\
    m \equiv -s \mod q
  \end{cases}

In [14]:
m4 = crt(int(-r),int(-s),int(p),int(q))
m4

7138284504269685812814890223560267091179245078270971466881304280429864636975

In [15]:
print(" m1: ", m1, "\n", "m2: ", m2, "\n", "m3: ", m3, "\n", "m4: ", m4, "\n")

 m1:  12341234 
 m2:  6125474802395748492703303010173044555221739825019323676217505696736377292704 
 m3:  1012809701873937320111587213387222535957505253251647790663798583693499685505 
 m4:  7138284504269685812814890223560267091179245078270971466881304280429864636975 



## Criação de Chaves

In [16]:
def gen_rand_keys(nbits):
    hbound = nbits
    lbound = nbits-1
    while 1:
        p = random_prime(2^hbound, 2^lbound, True)
        if (mod(p,4)==3) or (mod(p,8)==5):
            break
    while 1:
        q = random_prime(2^lbound, 2^(lbound-1), True)
        if (mod(q,4) == 3) or (mod(q,8) == 5):
            break
    PubKey=p*q
    PrivKey=(p,q)

    return PubKey,PrivKey

# Encriptação de Rabin

In [17]:
def rabin_encrypt(m, n):
    m = int(str(m) + str(m))

    if (m < n-1) and (m > 0):
        c = mod(m^2,n)
        return c
    else:
        print("Mensagem não válida")

# Decifração

In [18]:
def sqrt_mod(p,c):
    if(mod(p,4) == 3):
        Zp = IntegerModRing(p)
        rp = c^((p+1)/4)
        r = Zp(rp)
        return r
    if(mod(p,8) == 5):
        Zp = IntegerModRing(p)
        dp = c^((p-1)/4)
        d = Zp(dp)
        if(d == 1):
            rp = c^((p+3)/8)
            r = Zp(rp)


            return r
        if(d == p-1):
            rp = 2*c*((4*c)^((p-5)/8))
            r = Zp(rp)
            return r

In [19]:
def crt_sqrt(r,s,p,q):
    m1 = crt(int(r), int(s), int(p), int(q))
    m2 = crt(int(-r), int(s), int(p),int(q))
    m3 = crt(int(r), int(-s), int(p), int(q))
    m4 = crt(int(-r),int(-s),int(p),int(q))
    return m1, m2, m3, m4

In [20]:
def decrypt(c, PrivKey):
    p=PrivKey[0]
    q=PrivKey[1]
    r = sqrt_mod(p,c)
    s = sqrt_mod(q,c)
    return get_message(crt_sqrt(r,s,p,q))

In [21]:
def get_message(l):
    for i in l:
        num = str(i)
        comp = len(num)
        if(comp % 2 != 0):
            continue
        for j in range(comp/2):
            k = j+comp/2
            if(num[j] != num[k]):
                continue
            if(j == comp/2 - 1):
                return int(num[:(comp/2)])

In [22]:
keys = gen_rand_keys(1024)
PubKey = keys[0]
PrivKey = keys[1]
print("PubKey: ", PubKey, "\nPrivKey: ", PrivKey)

PubKey:  13904679250094574420735896503588575876311757506786425583798812996730023110774974528985572266980941881569020491007168552483222886355336661070431570742935528987410734597988488457308474884561194313096820929982114677725563366652691640914558543500793849393754480820890404707990103104563474026149641903002519960799294763592247836137120207652477254101098228661376804204810865836911397209975926345205882000600354615707523015322889650148633720150666439269556129924307610247635503745303998389788646172509647348065917514108017509384706731210640595982621447433817363944488526197943079392467362129112989595647448987020112923447387 
PrivKey:  (175835106337278340466772659285821182814971583791542703662296538061085819274781415479664794433316328245245227736907602844278930621087744317202499375322644753148230507769148232897418356834638583001959815421900547318104777036769994521187076888910486811025286354570533867558730288201035293905501423302599051815893, 79077947172979755423323531581961636680987102096920

In [23]:
c = rabin_encrypt(123456789, PubKey)
print("c: ",c,"\n")

c:  15241578780673678515622620750190521 



In [24]:
m = decrypt(c, PrivKey)
print("Mensagem: ", m)

Mensagem:  123456789


# Segurança

## Factorização

PubKey = n

$m^2 \equiv c \mod n$

## Escolha de Cifra

$m \in \mathbb{Z}^{*}_n$

$\iff$

$(m,n) = 1$

$c=m^2 \mod n$

Se: $y \not\equiv \pm m \mod n$

$(m-y,n) \in \text{factor(n)}$

Se: $y \not\equiv \pm m \mod n$